In [1]:
import sys
# from product import *
# from data_util.product import *
from data_util.mainCat import *

import pandas as pd
import random
import time
import re

import os
import hashlib
import struct
import subprocess
import collections
import tensorflow as tf
from tensorflow.core.example import example_pb2
import nltk

import sys
import shutil
import tqdm
import random

from copy import deepcopy
# from product import *

VOCAB_SIZE = 50000
CHUNK_SIZE = 1000  # num examples per chunk, for the chunked data

# Key word Attention DataSet 讀取

In [6]:
# main_cat = All_Electronics().getAttr()
main_cat = Pet_Supplies().getAttr()
# main_cat = Sports_Outdoors().getAttr()
# main_cat = Health_personal_Care().getAttr()

xlsx_path = "XLSX/main_cat/%s_key.xlsx"%(main_cat)
# df.to_csv(csv_path) #默认dt是DataFrame的一个实例，参数解释如下
# key_train_df.to_excel(csv_path, encoding='utf8')
orign_key_df = pd.read_excel(xlsx_path)
print(xlsx_path + " Read finished")
len(orign_key_df)
orign_key_df.head()

XLSX/main_cat/Pet Supplies_key.xlsx Read finished


,review_ID,review,summary,big_categories,main_cat,small_categories,lemm_review,lemm_summary,lemm_review_len,lemm_summary_len,overall,vote,total_keyword,FOP_sents,total_mention_features
0,1403740800,"It is not clear on the description, but this b...",Be aware of modification needed,Appliances,Pet Supplies,Parts & Accessories,not clear the description but this bracket mak...,<s> aware modification need </s>,82,5,3,4,"axle modify bracket unusable,wheel change",necessary to modify the axle for that and the ...,description hand blaster axle handle rest brac...
1,1428278400,"A lot to pay for the wall bracket, but it work...","A lot to pay for the wall bracket, but ...",Appliances,Pet Supplies,Parts & Accessories,lot to pay for the wall bracket but work.\nwor...,<s> lot to pay for the wall bracket but </s>,58,10,4,2,"bracket work,bracket wall",lot to pay for the wall bracket but work.,lot pay wall bracket wall blaster mount note f...
2,1394841600,Does what it is supposed to but ridiculously o...,okay - but ridiculous price,Appliances,Pet Supplies,Parts & Accessories,what suppose to but ridiculously over price fo...,<s> okay but ridiculous price </s>,32,6,3,8,"metal simple,metal bent",what suppose to but ridiculously over price fo...,piece metal suppose price simple gouge include...
3,1410739200,Worked well. I use to do water changes and wa...,Worked well. I use to do water changes and ...,Appliances,Pet Supplies,Water Filters,work well.\nuse to water change and surprise t...,<s> work well </s> \n <s> use to water change...,19,12,5,2,"suction surprise,pump hold",use to water change and surprise the suction c...,water pump suction surprise change cup glass s...
4,1406419200,This ribbon sucks! It tears easily and it's my...,This ribbon sucks! It tears easily and it's my...,"Arts, Crafts & Sewing",Pet Supplies,Ribbons,this ribbon suck.\ntear easily and fault miss ...,<s> this ribbon suck </s> \n <s> tear easily ...,73,12,1,3,tie tight,rip when try to tie bow and the bow ugly becau...,ribbon tear read satin satin plastic tight kno...


# Key word load

In [7]:
fn = 'FOP-View/%s_keywords2.txt' % (main_cat)
print('load %s keywords...' % (fn))
total_keywords = set()
with open(fn, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        k, v = line.split(":")
        total_keywords.add(k)

load FOP-View/Pet Supplies_keywords2.txt keywords...


FileNotFoundError: [Errno 2] No such file or directory: 'FOP-View/Pet Supplies_keywords2.txt'

# Total Opinion

In [ ]:
opinion_lexicon = {}
for filename in os.listdir('opinion-lexicon-English/'):      
    if "txt" not in filename: continue
    print(filename)
    with open('opinion-lexicon-English/'+filename,'r') as f_input:
        lexion = []
        for line in f_input:
            if line.startswith(";"):
                continue
            word = line.replace("\n","")
            if word != "" : lexion.append(word)
        pos = filename.replace(".txt","")
        opinion_lexicon[pos] = lexion

opinion_lexicon["total-words"] = opinion_lexicon["negative-words"] + opinion_lexicon["positive-words"]
print("total-words 已取得")

# Summary 資料清理

In [ ]:
import spacy
from collections import Counter
import en_core_web_sm
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

nlp = en_core_web_sm.load()

In [ ]:
def compose_summary(x):
    x = x.replace("\n","").replace("</s>","").replace("<s>","")
    x = "<s>" + x + "</s>"  
    x = " ".join([str(token) for token in nlp(x) if (" " not in str(token)) and \
                  (str(token).isalpha()) and \
                  (len(str(token)) > 1)   ])
    return x


def create_custom_tokenizer(nlp):
    prefix_re = re.compile(r'[0-9]\.')
    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search)

def calc_summary_len(x):
#     tokens = [token for token in nlp(x)]
#     print(tokens)
#     print([len(t) for t in tokens])
#     return len(tokens)
    return len(x.split(" "))

nlp.tokenizer = create_custom_tokenizer(nlp)

orign_key_df['lemm_summary'] = orign_key_df['lemm_summary'].apply(compose_summary)
orign_key_df['lemm_summary_len'] = orign_key_df['lemm_summary'].apply(calc_summary_len)


amount = len(orign_key_df)
print('Total data : %s'%(amount))

orign_key_df.head()


# review 多句合併

In [ ]:
from copy import deepcopy
def compose_review(x):
    x = eval(x)
    x = "\n".join(x)
    x = x.replace("\n"," ")    
    tokens = [str(token) for token in x.split(" ") if (" " not in str(token))and (str(token) == '.' or str(token).isalpha())]
    return " ".join(tokens)

def calc_review_len(x):
#     tokens = [token for token in nlp(x)]
#     print(tokens)
#     print([len(t) for t in tokens])
#     return len(tokens)
    return len(x.split(" "))
key_df = deepcopy(orign_key_df)
key_df['lemm_review'] = key_df['lemm_review'].apply(compose_review)
key_df['lemm_review_len'] = key_df['lemm_review'].apply(calc_review_len)
key_df.head()

In [ ]:
orign_key_df.head()

# 過濾不合適的訓練資料

In [ ]:
def to_words(text):
    keywords = set()
    for words in text.split(","):
        for word in words.split(" "):
            keywords.add(word)
    keywords = " ".join(keywords)
    return keywords

def calc_keyword_num(x):
    return len(x.split(" "))

# and(key_df.lemm_review_len>20)
flit_key_df = key_df[(key_df.lemm_summary_len>=4) ] # 過濾single word summary
flit_key_df = flit_key_df[(flit_key_df.lemm_review_len <= 1000) ] # 過濾single word summary

flit_key_df = flit_key_df.dropna(
    axis=0,     # 0: 对行进行操作; 1: 对列进行操作
    how='any'   # 'any': 只要存在 NaN 就 drop 掉; 'all': 必须全部是 NaN 才 drop 
    )


# FOP_keywords 資料整理

In [ ]:
flit_key_df['FOP_keywords'] = flit_key_df['total_keyword']
flit_key_df['FOP_keywords'] = flit_key_df['FOP_keywords'].apply(to_words)
flit_key_df['FOP_keywords_num'] = flit_key_df['FOP_keywords'].apply(calc_keyword_num)
flit_key_df = flit_key_df[(flit_key_df.FOP_keywords_num>=2) ] # 過濾single word summary
flit_key_df.head()

# Cheat Processing

In [ ]:
flit_key_df['Cheat'] = False 

# flit_key_df.head()
from tqdm import tqdm
# 非符號alpha word重疊數
with tqdm(total=len(flit_key_df)) as pbar:
    for i ,row in flit_key_df.iterrows():
        rev_tokens = set(row['lemm_review'].split(" "))
#         if 's' in rev_tokens: rev_tokens.remove('s')
#         if '.' in rev_tokens: rev_tokens.remove('.')
#         if 'i' in rev_tokens: rev_tokens.remove('i')
#         if 'a' in rev_tokens: rev_tokens.remove('a')
#         if 'the' in rev_tokens: rev_tokens.remove('the')
        
        summ_tokens = set(row['lemm_summary'].split(" "))
        key_sets = rev_tokens & summ_tokens & (total_keywords| set(opinion_lexicon["total-words"]))
        if len(key_sets) > 2: 
#             print(True)
            flit_key_df.loc[i,'Cheat'] = True

#         print(rev_tokens)
#         print(summ_tokens)
#         print([len(t) for t in summ_tokens])
        pbar.update(1)
    
flit_key_df = flit_key_df[(flit_key_df.Cheat == True) ] # 過濾single word summary
amount = len(flit_key_df)
print('Total data : %s'%(amount))
flit_key_df.head()


# TextRank_keywords 資料整理

In [ ]:
from summa import keywords as TextRank
from summa.summarizer import summarize
def textrank_keys(text):
    keywords1 = list()
    for words in TextRank.keywords(text).split('\n'):
        keywords1.extend(words.split(" "))
    keywords1 = set(keywords1)    
    
    return " ".join(list(keywords1))

def textrank_summ_keys(text): 
    keywords2 = list()
    for words in summarize(text, words=8).split('\n'):
        keywords2.extend(words.split(" "))
    keywords2 = set(keywords2)
    
    return " ".join(list(keywords2))

# flit_key_df['TextRank_keywords'] = flit_key_df['lemm_review'].apply(textrank_to_words)
# flit_key_df['TextRank_keywords_num'] = flit_key_df['TextRank_keywords'].apply(calc_num)
flit_key_df['TextRank_keywords'] = flit_key_df['FOP_keywords'] 
flit_key_df['TextRank_keywords_num'] = flit_key_df['FOP_keywords_num'] 
flit_key_df.loc[:,'TextRank_keywords'] = ''
flit_key_df.loc[:,'TextRank_keywords_num'] = 0
flit_key_df.head()



In [ ]:
from tqdm import tqdm
with tqdm(total=len(flit_key_df)) as pbar:
    for i ,row in flit_key_df.iterrows():
        TextRank_keywords = textrank_keys(row['lemm_review'])
    #     TextRank_keywords = textrank_summ_keys(row['lemm_review'])  
        num = calc_keyword_num(TextRank_keywords)
        flit_key_df.loc[i,'TextRank_keywords'] = TextRank_keywords
        flit_key_df.loc[i,'TextRank_keywords_num'] = num
        pbar.update(1)
        
flit_key_df.head()

# 輸出統計長度資料

In [ ]:
from matplotlib import pyplot as plt
if not os.path.exists('XLSX/statistic'):
    os.makedirs('XLSX/statistic')
with open('XLSX/statistic/%s_info.txt'%(main_cat),'w') as f:
    max_rev_len = flit_key_df['lemm_review_len'].max()
    min_rev_len = flit_key_df['lemm_review_len'].min()
    mean_rev_len = flit_key_df['lemm_review_len'].mean()
    median_rev_len = flit_key_df['lemm_review_len'].median()

    f.write('max_rev_len :%s \n'%(max_rev_len))
    f.write('min_rev_len :%s \n'%(min_rev_len))
    f.write('mean_rev_len :%s \n'%(mean_rev_len))
    f.write('median_rev_len :%s \n'%(median_rev_len))
    
    f.write('\n\n\n')
    max_summary_len = flit_key_df['lemm_summary_len'].max()
    min_summary_len = flit_key_df['lemm_summary_len'].min()
    mean_summary_len = flit_key_df['lemm_summary_len'].mean()
    median_summary_len = flit_key_df['lemm_summary_len'].median()

    f.write('max_summary_len :%s \n'%(max_summary_len))
    f.write('min_summary_len :%s \n'%(min_summary_len))
    f.write('mean_summary_len :%s \n'%(mean_summary_len))
    f.write('median_summary_len :%s \n'%(median_summary_len))
    
    f.write('\n\n\n')
    max_FOP_keywords_num = flit_key_df['FOP_keywords_num'].max()
    min_FOP_keywords_num = flit_key_df['FOP_keywords_num'].min()
    mean_FOP_keywords_num = flit_key_df['FOP_keywords_num'].mean()
    median_FOP_keywords_num = flit_key_df['FOP_keywords_num'].median()

    f.write('max_FOP_keywords_num :%s \n'%(max_FOP_keywords_num))
    f.write('min_FOP_keywords_num :%s \n'%(min_FOP_keywords_num))
    f.write('mean_FOP_keywords_num :%s \n'%(mean_FOP_keywords_num))
    f.write('median_FOP_keywords_num :%s \n'%(median_FOP_keywords_num))
    
    f.write('\n\n\n')
    max_TextRank_keywords_num = flit_key_df['TextRank_keywords_num'].max()
    min_TextRank_keywords_num = flit_key_df['TextRank_keywords_num'].min()
    mean_TextRank_keywords_num = flit_key_df['TextRank_keywords_num'].mean()
    median_TextRank_keywords_num = flit_key_df['TextRank_keywords_num'].median()

    f.write('max_TextRank_keywords_num :%s \n'%(max_TextRank_keywords_num))
    f.write('min_TextRank_keywords_num :%s \n'%(min_TextRank_keywords_num))
    f.write('mean_TextRank_keywords_num :%s \n'%(mean_TextRank_keywords_num))
    f.write('median_TextRank_keywords_num :%s \n'%(median_TextRank_keywords_num))

    

# plt.xlim(xmax = mean_rev_len)
# plt.ylim(ymax = flit_key_df['lemm_review_len'].value_counts().max())

flit_key_df['lemm_review_len'].value_counts().hist()
plt.savefig('XLSX/statistic/review_len_%s.png'%(main_cat))
plt.show()
plt.close()

# plt.xlim(xmax = max_summary_len)
# plt.ylim(ymax = flit_key_df['lemm_summary_len'].value_counts().max())
flit_key_df['lemm_summary_len'].value_counts().hist()
plt.savefig('XLSX/statistic/summary_len_%s.png'%(main_cat))
plt.show()
plt.close()

# plt.xlim(xmax = mean_keyword_num)
flit_key_df['FOP_keywords_num'].value_counts().hist()
plt.savefig('XLSX/statistic/FOP_keywords_num_%s.png'%(main_cat))
plt.show()
plt.close()

flit_key_df['TextRank_keywords_num'].value_counts().hist()
plt.savefig('XLSX/statistic/TextRank_keywords_num_%s.png'%(main_cat))
plt.show()
plt.close()


# 製作record bin檔

In [ ]:
import shutil
if os.path.exists('bin'):
    shutil.rmtree('/bin', ignore_errors=True)

if not os.path.exists('bin/main_cat/chunked'):
    os.makedirs('bin/main_cat/chunked')

makevocab = True
if makevocab:
    vocab_counter = collections.Counter()
    
# train_file
flit_key_train_df = flit_key_df.iloc[:int(amount*0.6)]

# test_file
flit_key_test_df = flit_key_df.iloc[int(amount*0.6)+1:int(amount*0.8)]

# vald_file
flit_key_valid_df = flit_key_df.iloc[int(amount*0.8)+1:]
sentence_start = "<s>"
sentence_end = "</s>"


def xlsx2bin(set_name,df):
    sents = []
    with open("bin/main_cat/%s.bin"%(set_name), 'wb') as file:
        i = 0
        for idx in tqdm(range(len(df))):
            series = df.iloc[idx]
            data_dict = series.to_dict()
            review_ID , big_categories , small_categories , \
            review , lemm_review , summary , lemm_summary , FOP_keywords ,TextRank_keywords = \
            data_dict['review_ID'],data_dict['big_categories'],data_dict['small_categories'],data_dict['review'],data_dict['lemm_review'], \
            data_dict['summary'],data_dict['lemm_summary'],data_dict['FOP_keywords'] ,data_dict['TextRank_keywords']
#             print(FOP_keywords)

            # save Embedding/word2Vec calculate sents
            for sent in nltk.sent_tokenize(lemm_review):
                sent = sent.replace("." ,"")
#                 sents.append(str(sent).split()) # 切分词汇 

            for sent in nltk.sent_tokenize(lemm_summary):
                sent = sent.replace(sentence_start ,"").replace(sentence_end ,"")
#                 sents.append(str(sent).split()) # 切分词汇 

            lemm_review = lemm_review.replace("\n","").replace("."," ")
            lemm_summary = lemm_summary.replace("\n","").replace("."," ")
            lemm_summary = sentence_start + ' '+ lemm_summary + ' ' + sentence_end
#             print(lemm_summary)
            # Write to tf.Example
            tf_example = example_pb2.Example()
            try:
                tf_example.features.feature['orign_review'].bytes_list.value.extend(
                    [tf.compat.as_bytes(review, encoding='utf-8')])

                tf_example.features.feature['orign_summary'].bytes_list.value.extend(
                    [tf.compat.as_bytes(summary, encoding='utf-8')])
                
                tf_example.features.feature['review'].bytes_list.value.extend(
                    [tf.compat.as_bytes(lemm_review, encoding='utf-8')])

                tf_example.features.feature['summary'].bytes_list.value.extend(
                    [tf.compat.as_bytes(lemm_summary, encoding='utf-8')]) 
        
                tf_example.features.feature['FOP_keywords'].bytes_list.value.extend(
                    [tf.compat.as_bytes(FOP_keywords, encoding='utf-8')]) 
            
                tf_example.features.feature['TextRank_keywords'].bytes_list.value.extend(
                    [tf.compat.as_bytes(TextRank_keywords, encoding='utf-8')]) 

                tf_example_str = tf_example.SerializeToString()
                str_len = len(tf_example_str)  
                file.write(struct.pack('q', str_len))
                file.write(struct.pack('%ds' % str_len, tf_example_str))
            except Exception as e:
#                 print(e)
                pass
    print(" %s finished... "%(file.name))
    return sents
    
    
sents1 = xlsx2bin('train',flit_key_train_df)
sents2 = xlsx2bin('test',flit_key_test_df)
sents3 = xlsx2bin('valid',flit_key_valid_df)

In [ ]:
with open("bin/main_cat/bin-info.txt",'w',encoding='utf-8') as f :
    f.write("train : %s\n"%(len(flit_key_train_df)))
    f.write("test : %s\n"%(len(flit_key_test_df)))
    f.write("valid : %s\n"%(len(flit_key_valid_df)))

# 分割record bin檔(1000為單位)

In [ ]:
def chunk_file(set_name, chunks_dir):
    in_file = 'bin/main_cat/%s.bin' % set_name
    reader = open(in_file, "rb")
    chunk = 0
    finished = False
    while not finished:
#         chunk_fname = os.path.join('bin', '/%s/%s_%03d.bin' % (chunks_dir,set_name, chunk))  # new chunk
        chunk_fname = '%s/%s/%s_%03d.bin' % (chunks_dir,set_name,set_name, chunk)
        with open(chunk_fname, 'wb') as writer:
            for _ in range(CHUNK_SIZE):
                len_bytes = reader.read(8)
                if not len_bytes:
                    finished = True
                    break
                str_len = struct.unpack('q', len_bytes)[0]
                example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
                writer.write(struct.pack('q', str_len))
                writer.write(struct.pack('%ds' % str_len, example_str))
            chunk += 1


def chunk_all(chunks_dir = 'bin/main_cat/chunked'):
    # Make a dir to hold the chunks
    
    # Chunk the data
    for set_name in ['train', 'valid', 'test']:
        if not os.path.isdir(os.path.join(chunks_dir,set_name)):
            os.mkdir(os.path.join(chunks_dir,set_name))
        print("Splitting %s data into chunks..." % set_name)
        chunk_file(set_name, chunks_dir)
    print("Saved chunked data in %s" % chunks_dir)
    
chunk_all()

In [ ]:
def main_valid():
    #Performing rouge evaluation on 1.9 lakh sentences takes lot of time. So, create mini validation set & test set by borrowing 15k samples each from these 1.9 lakh sentences
    bin_valid_chuncks = os.listdir('bin/main_cat/chunked/valid')
    bin_valid_chuncks.sort()
    if not os.path.exists('bin/main_cat/chunked/main_valid'):
        os.mkdir('bin/main_cat/chunked/main_valid')
        
    samples = random.sample(set(bin_valid_chuncks[:-1]), 2)      #Exclude last bin file; contains only 9k sentences
    valid_chunk, test_chunk = samples[0], samples[1]
    shutil.copyfile(os.path.join('bin/main_cat/chunked/valid', valid_chunk), os.path.join("bin/main_cat/chunked/main_valid", "valid_00.bin"))
    shutil.copyfile(os.path.join('bin/main_cat/chunked/valid', test_chunk), os.path.join("bin/main_cat/chunked/main_valid", "test_00.bin"))
main_valid()

# Embedding/word2Vec

In [ ]:
sentences = [] # total sentence
for idx in tqdm(range(len(orign_key_df))):
    series = orign_key_df.iloc[idx]
    data_dict = series.to_dict()
    lemm_review_sents , lemm_summary  = data_dict['lemm_review'],data_dict['lemm_summary'] 
    lemm_review_sents = eval(lemm_review_sents)
    for sent in lemm_review_sents:
        sent_tokens = sent.split(" ")
        tokens = [str(token) for token in sent.split() if (" " not in str(token))and (str(token) == '.' or str(token).isalpha())]
        sentences.append(tokens)     
    sentences.append(lemm_summary.split(" "))
print('word2Vec training sentence finished...')

In [ ]:
# sentences = sents1 + sents2 + sents3

In [ ]:
# 引入 word2vec
from gensim.models import word2vec
from glob import glob
import sys

import gensim
import torch
import torch.nn as nn
import torchsnooper
import os
import numpy as np

# 引入日志配置
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

vocab_count = 50000
# write vocab to file
if not os.path.exists('Embedding/main_cat/word2Vec'):
    os.makedirs('Embedding/main_cat/word2Vec')

In [ ]:
if os.path.exists("Embedding/main_cat/word2Vec/word2Vec.300d.txt"):

    w2vec = word2vec.Word2Vec(sentences, size=300, min_count=1,max_vocab_size=None,iter=100,
                              sorted_vocab=1,max_final_vocab=vocab_count)

    

    w2vec.wv.save_word2vec_format('Embedding/main_cat/word2Vec/word2Vec.300d.txt', binary=False)

    #保存模型，供日後使用
    # w2vec.save("Embedding/word2Vec/word2vec.model")  

In [ ]:
#模型讀取方式
# model = word2vec.Word2Vec.load("Embedding/word2Vec/word2vec.model")

wvmodel = gensim.models.KeyedVectors.load_word2vec_format(
    'Embedding/main_cat/word2Vec/word2Vec.300d.txt', binary=False, encoding='utf-8')

wvmodel.most_similar(u"player", topn=10)
# wvmodel.most_similar(['dvd','player','changer','machine','video'], topn=20)

In [ ]:
vocab_file = "bin/main_cat/word.vocab"

if os.path.exists(vocab_file):
    vocab_count = len(wvmodel.wv.index2entity)    

    print("Writing vocab file...")
    with open(vocab_file, 'w',encoding='utf-8') as writer:
        for word in wvmodel.wv.index2entity[:vocab_count]:
            # print(word, w2vec.wv.vocab[word].count)
            writer.write(word + ' ' + str(wvmodel.wv.vocab[word].count) + '\n') # Output vocab count
    print("Finished writing vocab file")

In [ ]:
word = wvmodel.wv.index2entity[25]
vector = wvmodel.wv.vectors[25]
print(word)
# print(vector)

In [ ]:
import torch
from data_util.data import Vocab
vocab_size = len(wvmodel.vocab) + 1


vocab = Vocab('bin/main_cat/word.vocab', vocab_size)

embed_size = 300
weight = torch.zeros(vocab_size, embed_size)

for i in range(len(vocab._id_to_word.keys())):
    try:
        vocab_word = vocab._id_to_word[i+4]
        w2vec_word = w2vec.wv.index2entity[i]
    except Exception as e :
        continue
    if i + 4 > vocab_size: break
#     print(vocab_word,w2vec_word)
    weight[i+4, :] = torch.from_numpy(w2vec.wv.vectors[i])
        
embedding = torch.nn.Embedding.from_pretrained(weight)
# requires_grad指定是否在训练过程中对词向量的权重进行微调
embedding.weight.requires_grad = True
embedding

In [ ]:
vocab.word2id('the')

# Embedding/glove

In [ ]:
vocab_size = len(wvmodel.vocab) + 1

vocab = Vocab('Embedding/main_cat/word2Vec/word2Vec.vocab', vocab_size)

with open("Embedding/glove/glove.6B.300d.txt", 'r',encoding='utf-8') as f :
#     print(vocab_size) 
    embed_size = 300
    weight = torch.zeros(vocab_size, embed_size)

    for line in f.readlines():
        values = line.split()
        word = values[0]
        if word not in vocab._word_to_id.keys(): continue
        vector = np.asarray(values[1:], "float32")
        wid = vocab.word2id(word)          
        weight[wid, :] = torch.from_numpy(vector)
        

embedding = torch.nn.Embedding.from_pretrained(weight)
# requires_grad指定是否在训练过程中对词向量的权重进行微调
embedding.weight.requires_grad = True
embedding        

# Embedding/Bert

In [ ]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
# BERT
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, do_basic_tokenize=True)
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
# model.eval()
model.embeddings.word_embeddings


# vocab = Vocab('Embedding/word2Vec/word2Vec.vocab', vocab_size)

# embed_size = 300
# weight = torch.zeros(vocab_size, embed_size)


# embedding = torch.nn.Embedding.from_pretrained(weight)
# # requires_grad指定是否在训练过程中对词向量的权重进行微调
# embedding.weight.requires_grad = True
# embedding        